In [ ]:
#Let's first import required Python packages.
#Importing required Python packages
import matplotlib.pylab as plt
import numpy as np
from sklearn.model_selection import ShuffleSplit, train_test_split
from sklearn import metrics
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier
from xgboost.sklearn import XGBClassifier  

import pandas as pd
np.random.seed(42)

/Users/rborra/anaconda/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [ ]:
# Reading the data into Python Dataframes
traincsv=pd.read_csv('train.csv', index_col='Id')
testcsv=pd.read_csv('test.csv', index_col='Id')

print traincsv.shape, testcsv.shape
print type(traincsv), type(testcsv)

In [ ]:
#Splitting the data in the ratio 80:20, the data is converted from Dataframes to numpy ndarray
X_train, X_test, y_train, y_test = train_test_split(traincsv.iloc[:,:-1].values, traincsv.iloc[:,-1:].values.ravel(), 
                                                    test_size=0.2)

print X_train.shape, X_test.shape, y_train.shape, y_test.shape
print type(X_train), type(X_test), type(y_train), type(y_test)

In [ ]:
#Fitting data to Classifiers, used Max Depth and Estimators based on experience and other factors
RFC=RandomForestClassifier(max_depth=6, n_estimators=100)
RFC.fit(X_train, y_train)

ETC= ExtraTreesClassifier(max_depth=6, n_estimators=100)
ETC.fit(X_train, y_train)

GBC= GradientBoostingClassifier(max_depth=6, n_estimators=100)
GBC.fit(X_train, y_train)

XGB= XGBClassifier(max_depth=6, n_estimators=100)
XGB.fit(X_train, y_train)

In [ ]:
# Data Stacking
estimators = [RFC, ETC, GBC, XGB]

# Creating empty lists to hold Predictions of the Classifiers
X_blend_train   =  []
X_blend_test    =  []
X_blend_testcsv =  []

print type(X_blend_train), type(X_blend_test), type(X_blend_testcsv)

# The lists are appended with the Classifier predictions, the list will have 4 arrays 
# which are predictions of 4 classifiers.
for i, est in enumerate(estimators):
    print i , est
    X_blend_train.append(est.predict(X_train))
    X_blend_test.append(est.predict(X_test))
    X_blend_testcsv.append(est.predict(testcsv.values))
    print len(X_blend_train), len(X_blend_test), len(X_blend_testcsv)
    print X_blend_train
    print X_blend_test
    print X_blend_testcsv

In [ ]:
print len(X_blend_train), len(X_blend_test), len(X_blend_testcsv)
#print X_blend_train.shape, X_blend_test.shape, X_blend_testcsv.shape
print type(X_blend_train), type(X_blend_test), type(X_blend_testcsv)

print X_blend_train
print X_blend_test
print X_blend_testcsv

# Transforming the Lists which have the Classifier predictions
X_blend_train = np.array(X_blend_train).T
X_blend_test = np.array(X_blend_test).T
X_blend_testcsv = np.array(X_blend_testcsv).T

print len(X_blend_train), len(X_blend_test), len(X_blend_testcsv)
print X_blend_train.shape, X_blend_test.shape, X_blend_testcsv.shape
print type(X_blend_train), type(X_blend_test), type(X_blend_testcsv)

print X_blend_train
print X_blend_test
print X_blend_testcsv

In [ ]:
XGB2= XGBClassifier(max_depth=6, n_estimators=100).fit(X_blend_train, y_train) 

#Make a copy of the test.csv 
temp=testcsv.copy()

#Run Predictions on test.csv 
temp['Cover_Type']=XGB2.predict(X_blend_testcsv) 

#Create Submissions csv file 
temp=temp['Cover_Type'] 
temp.to_csv('SampleSubmission17', header=True) 
print 'complete'

In [ ]:
# Public Score : 0.74605